In [15]:
from sklearn.model_selection import train_test_split
from pprint import pprint
import pandas as pd
import numpy as np
import sys
import os
import gc
import re
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

training_data = pd.read_csv("en_train.csv").sample(n=10000,random_state=40)
batch_size = 128
epochs = 5
max_num_features = 10
space_letter = 0
space_padded_tokens = []
max_data_size = len(training_data)

encoded_classes = pd.factorize(training_data['class'])
# print_data = 100000
x_data = []
labels = encoded_classes[1]
y_data = encoded_classes[0]
gc.collect()
count = 0

for before_value in training_data['before'].values:
    row = np.ones(max_num_features, dtype=int) * space_letter
    for before_value_char, i in zip(list(str(before_value)), np.arange(max_num_features)):
        row[i] = ord(before_value_char)
    count+=1
    x_data.append(row)
    
def make_flat_context_windows(data, pad_size, max_num_features, boundary_letter):
    pad = np.zeros(shape=max_num_features)
    #create array of pad arrays
    pads = [pad for _ in  np.arange(pad_size)]
    data = pads + data + pads
    flattened_context_windows = []
    for lower_bound in np.arange(len(data) - pad_size * 2):
        flattened_context_window = []
        
        context_window_size = pad_size * 2 + 1
        upper_bound = lower_bound + context_window_size
        context_window = data[lower_bound:upper_bound]
        
        for word in context_window:
            flattened_context_window.append([boundary_letter])
            flattened_context_window.append(word)
        flattened_context_window.append([boundary_letter])
        
        flattened_context_windows.append([int(word) for row in flattened_context_window for word in row])
    return flattened_context_windows


x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
x_data = np.array(make_flat_context_windows(x_data, pad_size = 1, max_num_features= max_num_features, boundary_letter=-1))
gc.collect()

x_train = np.array(x_data)
y_train = np.array(y_data)
gc.collect()

b = np.zeros((x_train.shape[0], x_train.shape[1]+1), dtype='O')
b[:,:-1] = x_train
b[:,-1] = np.array(training_data['before'].tolist())

c = np.zeros((x_train.shape[0], 2), dtype='O')
c[:,0] = np.array(y_train.tolist())
c[:,1] = np.array(training_data['after'].tolist())

x_train, x_valid, y_train, y_valid = train_test_split(b, c, test_size=0.2, random_state=2017)

num_classes = len(labels)
y_train1 = keras.utils.to_categorical(y_train[:,:-1], num_classes)
y_valid1 = keras.utils.to_categorical(y_valid[:,:-1], num_classes)
x_train1 = np.reshape(x_train[:,:-1], (x_train[:,:-1].shape[0], x_train[:,:-1].shape[1], 1))
x_valid1 = np.reshape(x_valid[:,:-1], (x_valid[:,:-1].shape[0], x_valid[:,:-1].shape[1], 1))

model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=((max_num_features * 3) + 4, 1)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(labels), activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train1, y_train1,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valid1, y_valid1))

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 9s 1ms/step - loss: 1.1097 - acc: 0.7007 - val_loss: 0.7616 - val_acc: 0.7550
Epoch 2/5
8000/8000 [==============================] - 5s 633us/step - loss: 0.6972 - acc: 0.7977 - val_loss: 0.4649 - val_acc: 0.8925
Epoch 3/5
8000/8000 [==============================] - 5s 622us/step - loss: 0.4934 - acc: 0.8855 - val_loss: 0.5682 - val_acc: 0.8555
Epoch 4/5
8000/8000 [==============================] - 5s 604us/step - loss: 0.4174 - acc: 0.9026 - val_loss: 0.3540 - val_acc: 0.9110
Epoch 5/5
8000/8000 [==============================] - 5s 644us/step - loss: 0.3892 - acc: 0.9073 - val_loss: 0.3090 - val_acc: 0.9230


In [16]:
a=model.predict(x_train1)
trainDatax=pd.DataFrame(x_train[:,:-1])
trainDatax['predict_class']=pd.DataFrame(np.argmax(a, axis=1).tolist())
trainDatax['before']=pd.DataFrame(x_train[:,-1])

trainDatay=pd.DataFrame(y_train)
trainDatay.columns = ['class','after']

trainDatay['change'] = 1
trainDatay['before'] = trainDatax['before']
trainDatay.loc[trainDatay.before == trainDatay.after, 'change'] = 0
trainDatax['predict_class'] = trainDatay['class']

x_train2 = [0 for x in range(len(labels))]
y_train2 = [0 for x in range(len(labels))]
model1 = [0 for x in range(len(labels))]
model2 = [0 for x in range(len(labels))]
for x in range(len(labels)):
    x_train2[x] = trainDatax[trainDatay['class']==x]
    y_train2[x] = trainDatay[trainDatay['class']==x]
    
    if x_train2[x].shape[0]==0:
        continue
    list1 = []
    for y in range(x_train2[x].shape[1]-2):
        list1.append(y)
    list1.append('predict_class')
    
    model1[x] = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10,10,10,10), activation='tanh',random_state=60)
    model1[x].fit(x_train2[x][list1],y_train2[x][['change']])

    y_d = []
    max_num_features=30
    for after_value in y_train2[x]['after'].values:
        row = np.ones(max_num_features, dtype=int) * space_letter
        for after_value_char, i in zip(list(str(after_value)), np.arange(max_num_features)):
            row[i] = ord(after_value_char)
        y_d.append(row)

    model2[x] = MLPRegressor(solver='lbfgs', hidden_layer_sizes=(100,100,100,100,100), activation='tanh',random_state=120)
    model2[x].fit(x_train2[x][list1],y_d)

In [17]:
def testing(X_test,y_test,labels):
    x_test1 = [0 for x in range(len(labels))]
    y_test1 = [0 for x in range(len(labels))]
    for x in range(len(labels)):
        print(x)
        x_test1[x] = X_test[X_test['predict_class']==x]
        y_test1[x] = y_test[X_test['predict_class']==x]

        if x_test1[x].shape[0]==0:
            continue
        
        list1 = []
        for y in range(x_test1[x].shape[1]-2):
            list1.append(y)
        list1.append('predict_class')
        
        x_test1[x]['change_predict'] = model1[x].predict(x_test1[x][list1]).astype(int)
        x_test1[x]['predict'] = x_test1[x]['change_predict']
        x_test1[x].loc[x_test1[x].change_predict == 0, 'predict'] = x_test1[x].loc[x_test1[x].change_predict == 0, 'before']
        if x_test1[x][x_test1[x]['change_predict']==1].shape[0]==0:
            continue

        p = model2[x].predict(x_test1[x][x_test1[x]['change_predict']==1][list1]).astype(int)
        p = np.array(p)
        
        row=0
        for y in x_test1[x][x_test1[x]['change_predict']==1].index.tolist():
            val = ""
            for col in range(p.shape[1]):
                try:
                    if p[row][col]>29 and p[row][col]<123:
                        val = val + chr(p[row][col])
                except:
                    continue
            
            x_test1[x].loc[y,'predict'] = val
            row=row+1
        
        
    input1=x_test1[0]
    output1=y_test1[0]
    for x in range(1,len(labels)):
        input1 =pd.concat([input1,x_test1[x]])
        output1 =pd.concat([output1,y_test1[x]])
    
    input1 = input1[['before','predict','predict_class']]
    output1 = output1['after']
    
    print("Accuracy : "+str(input1[input1['predict']==output1].shape[0]*100/input1.shape[0]))
    return input1,output1

In [18]:
print("Training : ")
print("Number of instances : "+str(trainDatax.shape[0]))
input1,output1=testing(trainDatax,trainDatay,labels)
print("Testing : ")

a=model.predict(x_valid1)
testDatax=pd.DataFrame(x_valid[:,:-1])
testDatax['predict_class']=pd.DataFrame(np.argmax(a, axis=1).tolist())
testDatax['before']=pd.DataFrame(x_valid[:,-1])

testDatay=pd.DataFrame(y_valid)
testDatay.columns = ['class','after']

print("Number of instances : "+str(testDatax.shape[0]))
input2,output2=testing(testDatax,testDatay,labels)

Training : 
Number of instances : 8000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Accuracy : 93.1
Testing : 
Number of instances : 2000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Accuracy : 92.3


In [14]:
input1[input1['predict_class']>1]

,before,predict,predict_class
273,$21.98,opiSlWSmhmAG[IBVZZON3YK[][9>>=9FHG=;54#%,2
319,$1.50,opiSlWSmhmAG[IBVZZON3YK[][9>>=9FHG=;54#%,2
1817,$1,opiSlWSmhmAG[IBVZZON3YK[][9>>=9FHG=;54#%,2
3267,"$29,066",opiSlWSmhmAG[IBVZZON3YK[][9>>=9FHG=;54#%,2
5501,HK$330 million,opiSlWSmhmAG[IBVZZON3YK[][9>>=9FHG=;54#%,2
6416,"$40,521",opiSlWSmhmAG[IBVZZON3YK[][9>>=9FHG=;54#%,2
25,2003,onlRph^lJhdeRaXG?=3$,3
46,1879,onlRph^lJhdeRaXG?=3$,3
92,1922,onlRph^lJhdeRaXG?=3$,3
121,"December 6, 2010",nhkMhfgdcdajdaVe\TacZZ]UUSOFD@58231.'# ,3
